In [1]:
import urllib2
import urllib
import StringIO
import ast
import pandas as pd
from IPython.display import Image

SQL_SOURCE = 'https://mjs639.carto.com/api/v2/sql?q='


def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return response.read()

SyntaxError: invalid syntax (<ipython-input-1-65caf769b286>, line 25)

In [ ]:
#Select trips under 3 hours in length, ordered from longest to shortest

task1 = '''
SELECT * FROM citibike
WHERE tripduration<10800
ORDER BY tripduration DESC
LIMIT 10
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task1)), sep=',')

In [ ]:
#Finding the average trip duration and birth year

task2 = '''
SELECT AVG(tripduration) as avt, AVG(birth_year) as avb
FROM citibike
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task2)), sep=',')

In [ ]:
#Isolating trips taken on February 2nd, 2015

task3 = '''
SELECT *
FROM citibike
WHERE starttime >= '2015-02-02 00:00'
  AND starttime < '2015-02-03 00:00'
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task3)), sep=',')

In [ ]:
#Finding the average trip duration for weekdays

task4 = '''
SELECT AVG(tripduration)
FROM citibike
WHERE extract(DOW from starttime) NOT IN (0,6) 
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task4)), sep=',')

In [ ]:
#Identifying every station used

task5 = '''
SELECT start_station_latitude, start_station_longitude
FROM citibike
GROUP BY start_station_latitude, start_station_longitude
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task5)), sep=',')

In [ ]:
#Counting the number of trips originating at each unique station

task6 = '''
SELECT start_station_id, COUNT(*)
FROM citibike
GROUP BY start_station_id, 
start_station_latitude, start_station_longitude
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task6)), sep=',')

In [ ]:
#Creating Geom Coordinates for each station

task7 = '''
SELECT CDB_LatLng(start_station_latitude, start_station_longitude) as the_geom
FROM citibike
GROUP BY start_station_id, 
start_station_latitude, start_station_longitude
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task7)), sep=',')

In [ ]:
#Connecting each station with its coordinates, and mapping them

task8 = '''
SELECT CDB_TransformToWebmercator(
           CDB_LatLng(start_station_latitude, 
                      start_station_longitude)) as the_geom_webmercator,
       MIN(cartodb_id) as cartodb_id
FROM citibike
GROUP BY start_station_id, 
start_station_latitude, start_station_longitude
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task8)), sep=',')

In [ ]:
#Map from Task 8

Image("https://raw.githubusercontent.com/mjs639/PUI2017_mjs639/master/HW7_mjs639/Screen%20Shot%202017-11-02%20at%207.13.57%20PM.png")

### Figure 1 shows every unique Citi Bike station active during February, 2015

In [ ]:
#Mapping stations within 500 meters of Times Square

task9 = '''
SELECT CDB_TransformToWebmercator(
           CDB_LatLng(start_station_latitude, 
                      start_station_longitude)) as the_geom_webmercator,
       MIN(cartodb_id) as cartodb_id 
FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude, 
                            start_station_longitude)::geography,
                 CDB_LatLng(40.7577, -73.9857)::geography,
                 500)
GROUP BY start_station_id, 
start_station_latitude, start_station_longitude
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task9)), sep=',')

In [ ]:
#Map from Task 9

Image("https://raw.githubusercontent.com/mjs639/PUI2017_mjs639/master/HW7_mjs639/Screen%20Shot%202017-11-02%20at%207.20.58%20PM.png")

### Figure 2 shows the seven stations that are located within 500 meters of the heart of Times Square, identified as the point at coordinates (40.7577, -73.9857)

In [ ]:
#Determining the average trip duration originating at each station within 500 meters of Times Square, 
#and representing this information on the map

task10 = '''
SELECT CDB_TransformToWebmercator(
           CDB_LatLng(start_station_latitude, 
                      start_station_longitude)) as the_geom_webmercator,
       MIN(cartodb_id) as cartodb_id, 
       AVG(tripduration) as ta
FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude, 
                            start_station_longitude)::geography,
                 CDB_LatLng(40.7577, -73.9857)::geography,
                 500)
GROUP BY start_station_id, 
start_station_latitude, start_station_longitude
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task10)), sep=',')

In [ ]:
#Map from Task 10

Image("https://raw.githubusercontent.com/mjs639/PUI2017_mjs639/master/HW7_mjs639/Screen%20Shot%202017-11-02%20at%207.27.46%20PM.png")

### Figure 3 represents the average trip duration for trips originating at the seven Times Square stations. The larger the dot, the higher the average is.

In [ ]:
#Representing every trip starting at the stations within 500 meters of Times Square and connecting them to the 
#end stations using a line

task11 = '''
SELECT ST_Makeline(
       CDB_TransformToWebmercator(
           CDB_LatLng(start_station_latitude, 
                      start_station_longitude)),
       CDB_TransformToWebmercator(
           CDB_LatLng(end_station_latitude, 
                      end_station_longitude))
       ) as the_geom_webmercator,          
       MIN(cartodb_id) as cartodb_id 
FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude, 
                            start_station_longitude)::geography,
                 CDB_LatLng(40.7577, -73.9857)::geography,
                 500)
GROUP BY start_station_id, start_station_latitude, start_station_longitude,
         end_station_id, end_station_latitude, end_station_longitude
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task11)), sep=',')

In [ ]:
#Map from Task 11

Image("https://raw.githubusercontent.com/mjs639/PUI2017_mjs639/master/HW7_mjs639/Screen%20Shot%202017-11-02%20at%207.32.16%20PM.png")

### Figure 4 shows the end point of every trip originating from the Times Square Seven. Each trip is represented by a line from the origin to the destintion.

## Ash - Scoer # 100. Extra Credit 0